In [1]:
import pandas as pd
import numpy as np

def format_func(value, tick_number):
    if value == 0:
        return "0"
    elif value >= 1000:
        return f"{int(value/1000)}k"
    else:
        return str(int(value))

def get_metric_values(df, input_type, metric_name):
    if input_type not in df["input_type"].unique():
        raise ValueError(f"'{input_type}' is not a valid input_type. "
                         f"Available: {df['input_type'].unique().tolist()}")
    if metric_name not in df["metric_name"].unique():
        raise ValueError(f"'{metric_name}' is not a valid metric_name. "
                         f"Available: {df['metric_name'].unique().tolist()}")
    
    metric_values = df[(df["input_type"] == input_type) &
                       (df["metric_name"] == metric_name)]["value"]
    return metric_values.values

def read_metrics(experiment_key: str) -> [pd.DataFrame, int]:
    try:
        experiment_metrics = pd.read_csv(f"loggs/{experiment_key}/metrics.csv")
        num_loged_epochs = experiment_metrics["epoch"].max() + 1
        return experiment_metrics, num_loged_epochs
    except FileNotFoundError:
        print(f"Error: The directory for experiment '{experiment_key}' was not found.")
        print("Please make sure you have run the short experiments first.")
        return None, 0

def get_model_norm(results):
    return results[results["metric_name"] == "weights_l2"] \
        .groupby("epoch")["value"] \
        .apply(lambda x: np.sqrt((x**2).sum())).values

## Figure X: Short Baseline vs. DP

In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd

# Experiment keys for the short (2000 epoch) runs.
baseline_experiment_key = "add_mod|num_epochs-2000|train_fraction-0.4|log_frequency-500|lr-0.0005|batch_size-5107|cross_entropy_dtype-float16|adam_epsilon-1e-30"
dp_experiment_key = "add_mod|lr-0.0005|num_epochs-2000|log_frequency-500|train_fraction-0.4|cross_entropy_dtype-float16|adam_epsilon-1e-30|use_dp-True|target_epsilon-1.0"

baseline_metrics, num_logged_epochs = read_metrics(baseline_experiment_key)
dp_metrics, _ = read_metrics(dp_experiment_key)

if baseline_metrics is not None and dp_metrics is not None:
    log_frequency = 500
    epochs = range(0, log_frequency * (num_logged_epochs), log_frequency)

    baseline_train_accuracy = get_metric_values(baseline_metrics, "train", "accuracy")
    baseline_test_accuracy = get_metric_values(baseline_metrics, "test", "accuracy")

    dp_train_accuracy = get_metric_values(dp_metrics, "train", "accuracy")
    dp_test_accuracy = get_metric_values(dp_metrics, "test", "accuracy")

    fig, ax = plt.subplots(figsize=(6, 5))

    ax.plot(epochs, baseline_train_accuracy, linewidth=5, label='Baseline Training accuracy', color='blue', linestyle='--')
    ax.plot(epochs, baseline_test_accuracy, linewidth=5, label='Baseline Test accuracy', color='blue')

    ax.plot(epochs, dp_train_accuracy, linewidth=5, label='DP Training accuracy', color='red', linestyle='--')
    ax.plot(epochs, dp_test_accuracy, linewidth=5, label='DP Test accuracy', color='red')

    ax.set_xlabel('Epoch', fontsize=18)
    ax.set_ylabel('Accuracy (%)', fontsize=18)

    ax.grid(alpha=0.5)
    ax.legend(fontsize=14, loc="lower right")

    ax.xaxis.set_major_formatter(FuncFormatter(format_func))

    ax.set_xticks(epochs[::2])
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.tight_layout()
    plt.show()


Error: The directory for experiment 'add_mod|lr-0.0005|num_epochs-2000|log_frequency-500|train_fraction-0.4|cross_entropy_dtype-float16|adam_epsilon-1e-30|use_dp-True|target_epsilon-1.0' was not found.
Please make sure you have run the short experiments first.
